# Hyperparameter Search

In [1]:
%pwd

'D:\\15_Article\\PyCode_v241111'

In [2]:
%cd ..

D:\15_Article


In [3]:
!dir

 Volume in drive D has no label.
 Volume Serial Number is 0E99-E4FB

 Directory of D:\15_Article

11/18/2024  11:05 AM    <DIR>          .
11/18/2024  11:05 AM    <DIR>          ..
11/11/2024  05:55 PM               364 .gitignore
11/18/2024  11:05 AM    <DIR>          .virtual_documents
11/12/2024  11:06 AM    <DIR>          01_Data
11/11/2024  05:55 PM    <DIR>          02_RCode
11/11/2024  05:55 PM               218 03_RStudio.Rproj
11/11/2024  05:55 PM    <DIR>          04_Figure
11/11/2024  05:55 PM    <DIR>          05_Manuscript
11/12/2024  11:02 AM    <DIR>          06_PyCode
11/11/2024  05:55 PM    <DIR>          08_ShScript
11/11/2024  05:55 PM            11,558 LICENSE
11/18/2024  11:08 AM    <DIR>          PyCode_v241111
11/11/2024  05:55 PM             4,122 README.md
               4 File(s)         16,262 bytes
              10 Dir(s)  372,727,357,440 bytes free


## Import Package 

In [25]:
from glob import glob
from joblib import dump, load
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer

### Function

In [5]:
### X and y
def getXandY(Output_Vari):
    y_list = glob("01_Data/*_y_" + Output_Vari + "*.csv")
    y = pd.read_csv(y_list[0], index_col=0)
    y = y.iloc[:,0].to_numpy()
    X_list = glob("01_Data/*_X_" + Output_Vari + "*.csv")
    X = pd.read_csv(X_list[0], index_col=0)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, 
                                                        random_state=1)
    return X_train, X_test, y_train, y_test

## Experiment for Happiness

### Load Data

In [6]:
Output_Vari = "Happinessoverall"

In [7]:
X_train, X_test, y_train, y_test = getXandY(Output_Vari)

In [8]:
X_train.head()

,year,lat,lon,female,age,high_stress,low_stress,easy_to_relax,good_for_living,live_environment_satefy,...,housewife,retired,unemployed,college_no_diploma,bachelor,master,phd,income_indiv,NDVI,NTL
329671,2016,37.183964,138.256897,1,51,4,5,3,4,3,...,0,0,0,0,0,0,0,3.5,29.553186,4.315041
24633,2015,37.239090,140.353348,1,26,5,5,3,3,4,...,0,0,0,1,0,0,0,1.0,40.441441,2.005019
279845,2016,35.528252,140.185822,0,67,2,1,4,4,3,...,0,0,0,0,1,0,0,1.0,53.940829,7.728996
269295,2016,33.566044,130.342041,1,35,1,3,4,4,3,...,1,0,0,1,0,0,0,1.0,30.014896,17.481237
232515,2015,33.619823,130.515259,1,32,4,4,2,4,3,...,1,0,0,0,1,0,0,2.5,55.156878,5.779940


In [29]:
X_train.describe()

,year,lat,lon,female,age,high_stress,low_stress,easy_to_relax,good_for_living,live_environment_satefy,...,housewife,retired,unemployed,college_no_diploma,bachelor,master,phd,income_indiv,NDVI,NTL
count,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,...,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000,344855.000000
mean,2015.656540,35.708914,137.514520,0.352612,48.967114,3.172557,3.606846,3.293581,3.826623,2.953085,...,0.086381,0.064410,0.046147,0.209624,0.437245,0.047295,0.013539,4.686452,36.247119,18.599653
std,0.771443,2.131133,3.052344,0.477784,11.731917,1.166244,1.060261,1.020122,0.897251,0.761716,...,0.280927,0.245482,0.209804,0.407041,0.496047,0.212270,0.115567,3.903289,13.664925,15.865111
min,2015.000000,24.301767,123.762947,0.000000,17.000000,1.000000,1.000000,1.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,4.860561,0.013894
25%,2015.000000,34.747284,135.545242,0.000000,41.000000,2.000000,3.000000,3.000000,3.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,26.247235,6.048134
50%,2015.000000,35.477516,138.984985,0.000000,49.000000,3.000000,4.000000,4.000000,4.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.500000,34.654246,14.545732
75%,2016.000000,35.809013,139.715393,1.000000,57.000000,4.000000,4.000000,4.000000,4.000000,3.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,6.500000,44.238307,26.747059
max,2017.000000,45.511021,145.742340,1.000000,101.000000,5.000000,5.000000,5.000000,5.000000,4.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,30.000000,87.709065,88.380999


In [9]:
X_train.columns

Index(['year', 'lat', 'lon', 'female', 'age', 'high_stress', 'low_stress',
       'easy_to_relax', 'good_for_living', 'live_environment_satefy',
       'community_attachment', 'income', 'self_reported_health', 'student',
       'worker', 'company_owner', 'government_officer', 'self_employed',
       'professional', 'housewife', 'retired', 'unemployed',
       'college_no_diploma', 'bachelor', 'master', 'phd', 'income_indiv',
       'NDVI', 'NTL'],
      dtype='object')

In [10]:
X_train.shape

(344855, 29)

In [11]:
X_test.shape

(38318, 29)

In [12]:
X = pd.concat([X_train, X_test])

In [13]:
X.shape

(383173, 29)

In [14]:
y = np.concatenate([y_train, y_test])

In [15]:
y.shape

(383173,)

### Run Test Model

In [16]:
param_space = {
    'n_estimators': Integer(100, 2000),
    'max_depth': Integer(3, 16),
    'max_samples': Real(0.5, 1.0),
    'min_samples_split':  Integer(2, 32)
}

In [17]:
rf_reg =RandomForestRegressor(n_jobs = 6)

In [18]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [19]:
rkfcv = RandomRunNFoldsKFold(n_splits=10, run_splits=3, random_state=42)

In [20]:
bayes_search = BayesSearchCV(
    estimator=rf_reg,
    search_spaces=param_space,
    n_iter=50,
    scoring='r2',
    cv=rkfcv,
    n_jobs = 1,
    n_points = 1,
    verbose=2,
    random_state=42,
    return_train_score = True
)

In [21]:
bayes_search.fit(X, y)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=8, max_samples=0.8638628715886625, min_samples_split=30, n_estimators=700; total time= 3.3min
[CV] END max_depth=8, max_samples=0.8638628715886625, min_samples_split=30, n_estimators=700; total time= 3.3min
[CV] END max_depth=8, max_samples=0.8638628715886625, min_samples_split=30, n_estimators=700; total time= 3.4min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=14, max_samples=0.9416576386904312, min_samples_split=11, n_estimators=1907; total time=15.9min
[CV] END max_depth=14, max_samples=0.9416576386904312, min_samples_split=11, n_estimators=1907; total time=15.1min
[CV] END max_depth=14, max_samples=0.9416576386904312, min_samples_split=11, n_estimators=1907; total time=15.0min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=9, max_samples=0.9593612608346885, min_samples_split=5, n_estimators=923; total time= 5.4min
[CV] END max_depth=9, max_s

D:\anaconda3\envs\ML\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[CV] END max_depth=10, max_samples=0.9600046132186582, min_samples_split=17, n_estimators=1692; total time= 9.8min
[CV] END max_depth=10, max_samples=0.9600046132186582, min_samples_split=17, n_estimators=1692; total time= 9.8min
[CV] END max_depth=10, max_samples=0.9600046132186582, min_samples_split=17, n_estimators=1692; total time= 9.9min
Fitting 3 folds for each of 1 candidates, totalling 3 fits


D:\anaconda3\envs\ML\lib\site-packages\numpy\ma\core.py:2846: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


[CV] END max_depth=15, max_samples=0.8503107223106829, min_samples_split=28, n_estimators=890; total time= 7.0min
[CV] END max_depth=15, max_samples=0.8503107223106829, min_samples_split=28, n_estimators=890; total time= 6.6min
[CV] END max_depth=15, max_samples=0.8503107223106829, min_samples_split=28, n_estimators=890; total time= 6.7min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=3, max_samples=0.9089150098318758, min_samples_split=24, n_estimators=584; total time= 1.0min
[CV] END max_depth=3, max_samples=0.9089150098318758, min_samples_split=24, n_estimators=584; total time= 1.1min
[CV] END max_depth=3, max_samples=0.9089150098318758, min_samples_split=24, n_estimators=584; total time= 1.0min
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END max_depth=16, max_samples=0.5132998647512224, min_samples_split=4, n_estimators=1943; total time=11.1min
[CV] END max_depth=16, max_samples=0.5132998647512224, min_samples_split=4, n_estimator

BayesSearchCV(cv=RandomRunNFoldsKFold(n_splits=10, random_state=42, run_splits=None),
              estimator=RandomForestRegressor(n_jobs=6), random_state=42,
              return_train_score=True, scoring='r2',
              search_spaces={'max_depth': Integer(low=3, high=16, prior='uniform', transform='normalize'),
                             'max_samples': Real(low=0.5, high=1.0, prior='uniform', transform='normalize'),
                             'min_samples_split': Integer(low=2, high=32, prior='uniform', transform='normalize'),
                             'n_estimators': Integer(low=100, high=2000, prior='uniform', transform='normalize')},
              verbose=2)

In [22]:
CV_result = bayes_search.cv_results_

In [23]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_samples,param_min_samples_split,param_n_estimators,params,split0_test_score,...,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,rank_train_score
27,660.487450,13.732494,2.349241,0.011452,16,0.500000,2,2000,"{'max_depth': 16, 'max_samples': 0.5, 'min_sam...",0.380535,...,0.377377,0.377806,0.002076,1,0.541761,0.542862,0.543347,0.542657,0.000663,1
29,682.229658,15.265986,2.357944,0.020112,16,0.500000,3,1998,"{'max_depth': 16, 'max_samples': 0.5, 'min_sam...",0.380469,...,0.377337,0.377715,0.002111,2,0.539502,0.540509,0.541119,0.540377,0.000667,3
10,676.124750,11.325201,2.273194,0.052037,16,0.513300,4,1943,"{'max_depth': 16, 'max_samples': 0.51329986475...",0.380381,...,0.377333,0.377670,0.002090,3,0.537647,0.538560,0.539317,0.538508,0.000683,4
31,647.346344,9.027588,2.206379,0.029000,16,0.500000,12,2000,"{'max_depth': 16, 'max_samples': 0.5, 'min_sam...",0.380523,...,0.377246,0.377609,0.002246,4,0.507565,0.508463,0.508951,0.508326,0.000574,8
36,644.492992,3.006706,2.131648,0.038649,16,0.500000,16,2000,"{'max_depth': 16, 'max_samples': 0.5, 'min_sam...",0.380592,...,0.377010,0.377591,0.002251,5,0.497361,0.498275,0.498479,0.498038,0.000486,10
23,635.578019,4.366977,1.979740,0.031294,16,0.500000,32,2000,"{'max_depth': 16, 'max_samples': 0.5, 'min_sam...",0.380583,...,0.377018,0.377560,0.002279,6,0.470913,0.471645,0.471928,0.471495,0.000427,16
19,613.712762,10.292857,1.991988,0.006388,15,0.500000,2,2000,"{'max_depth': 15, 'max_samples': 0.5, 'min_sam...",0.380393,...,0.377114,0.377542,0.002174,7,0.511448,0.512314,0.512671,0.512144,0.000514,7
16,666.896071,10.430577,1.986580,0.059622,15,0.500000,32,2000,"{'max_depth': 15, 'max_samples': 0.5, 'min_sam...",0.380264,...,0.376744,0.377278,0.002253,8,0.457644,0.458381,0.458641,0.458222,0.000422,22
40,556.417001,11.436391,1.564332,0.017250,15,0.618845,5,1514,"{'max_depth': 15, 'max_samples': 0.61884536508...",0.379449,...,0.376564,0.376767,0.002112,9,0.513565,0.514535,0.515147,0.514416,0.000651,6
46,716.828865,3.177451,1.982937,0.033345,16,0.636264,23,1874,"{'max_depth': 16, 'max_samples': 0.63626434770...",0.379578,...,0.376453,0.376762,0.002184,10,0.495059,0.495881,0.496311,0.495750,0.000520,11


In [26]:
dump(bayes_search, 'Results/BayesSearch50iter.joblib')

['Results/BayesSearch50iter.joblib']

In [28]:
pd.DataFrame(CV_result).sort_values(by='rank_test_score', ascending=True).to_csv('Results/BayesSearch50iter.csv')

### Test Perfromance

In [30]:
bayes_search.best_params_

OrderedDict([('max_depth', 16),
             ('max_samples', 0.5),
             ('min_samples_split', 2),
             ('n_estimators', 2000)])

In [31]:
rf_reg_final =RandomForestRegressor(n_jobs = 6, **bayes_search.best_params_)
rf_reg_final.fit(X_train, y_train)

RandomForestRegressor(max_depth=16, max_samples=0.5, n_estimators=2000,
                      n_jobs=6)

In [32]:
# Predictions
y_pred = rf_reg_final.predict(X_test)

In [33]:
y_pred

array([3.86886447, 2.7416812 , 3.71272488, ..., 4.18871424, 3.22898312,
       3.89751971])

In [34]:
y_pred_categorical = np.where(y_pred < 1.5, 1,
                       np.where(y_pred < 2.5, 2,
                         np.where(y_pred < 3.5, 3,
                           np.where(y_pred < 4.5, 4, 5))))

y_pred_categorical

array([4, 3, 4, ..., 4, 3, 4])

In [36]:
r2_score(y_test, y_pred_categorical)

0.29472796658377765

In [39]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred_categorical)

np.float64(0.46675191815856776)